## Import required libraries for Subgrounds interface

In [1]:
# Install all required libraries for the subgrounds layer
from datetime import datetime
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds

## Create an instance of subgrounds and connect to the subgraph api of choice (in this case, klimaDAO Subgraph)

In [2]:
# instantiate subgrounds
sg = Subgrounds()
# connect to the klimaDAO subgraph API
klimaDAO = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/cujowolf/klima-graph')

## In this section, we construct the subgrounds synthetic fields and secondary transformations that we need for our dashboard

In [4]:
klimaDAO.ProtocolMetric.datetime = SyntheticField(
    lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    SyntheticField.STRING,
    klimaDAO.ProtocolMetric.timestamp,
)

In [5]:
klimaDAO.ProtocolMetric.staked_supply_percent = SyntheticField(
    lambda sklima_supply, total_supply: 100 * sklima_supply/total_supply,
    SyntheticField.FLOAT,
    [
        klimaDAO.ProtocolMetric.sKlimaCirculatingSupply,
        klimaDAO.ProtocolMetric.totalSupply
    ],
    default = 100.0
)

In [6]:
klimaDAO.ProtocolMetric.unstaked_supply_percent = 100 - klimaDAO.ProtocolMetric.staked_supply_percent

In [8]:
klimaDAO.ProtocolMetric.rfv_per_klima = SyntheticField(
    lambda treasury_rfv, total_supply: treasury_rfv / total_supply if treasury_rfv / total_supply > 1 else 0,
    SyntheticField.FLOAT,
    [
        klimaDAO.ProtocolMetric.treasuryRiskFreeValue,
        klimaDAO.ProtocolMetric.totalSupply
    ]
)


## In this section, we construct the subgrounds queries

In [9]:
# Create a function to pull immediate (latest) data points for any desired metrics you query
def immediate (sg: Subgrounds, fpath: FieldPath):
    data = sg.execute(sg.mk_request([fpath]))
    return fpath.extract_data(data)[0]

In [12]:
protocol_metrics_1year = klimaDAO.Query.protocolMetrics(
    orderBy = klimaDAO.ProtocolMetric.timestamp,
    orderDirection = 'desc',
    first = 365
)


In [13]:
last_metric = klimaDAO.Query.protocolMetrics(
    orderBy = klimaDAO.ProtocolMetric.timestamp,
    orderDirection = 'desc',
    first = 1
)


## Now we are ready to interface our Subgrounds layer with the web app layer!

### Steps:
- Import the Subgrounds fields and queries - Everything we created in the subgrounds layer
- Import dash core components - Library of core dash components
- Import dash bootstrap components - Library of standard bootstrap styling components for dash
- Import dash plotly components - Library of data visualization components

In [14]:
import dash
import dash_bootstrap_components as dbc
from dash import html
from subgrounds.dash_wrappers import Graph
from subgrounds.plotly_wrappers import Figure, Scatter, Bar, Figure
from millify import millify

In [16]:
from jupyter_dash import JupyterDash

# dashboard
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H4('Market Cap'),
    html.Div([
        Graph(
            Figure(
                subgrounds=sg,
                traces=[
                    Scatter(
                        name='Market Cap',
                        x = protocol_metrics_1year.datetime,
                        y = protocol_metrics_1year.marketCap,
                        stackgroup='one'
                    )
                ]
            )
        )
    ])
])
app.run_server(mode='inline', port = 8091, dev_tools_ui=True, dev_tools_hot_reload=True, threaded=True)

/Users/tachikoma000/.local/share/virtualenvs/subgrounds_app_demo-UbIKUZWV/lib/python3.10/site-packages/werkzeug/serving.py:700: ResourceWarning:

unclosed <socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 0)>



### Now let's build an app!

#### Steps
- Create your Subgrounds layer in a separate py file: This is where you interface with the subgraph, create transformations and define your query types
- Create your app file: This is where you construct your dashboard and insert into an app container


### Follow me!